In [17]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from catboost import CatBoostClassifier


In [18]:

# Load the data
train = pd.read_csv('Train.csv').dropna(axis=0)  # Read in train, ignoring one row with missing data
test = pd.read_csv('Test.csv').fillna('')  # Read in test

In [19]:
train.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [20]:
# Split data into training and validation sets
df_train, df_valid = train_test_split(train, test_size=0.2, random_state=42)

In [21]:
# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

In [22]:
# Transform text data into TF-IDF features
X_train = vectorizer.fit_transform(df_train['safe_text'])
X_valid = vectorizer.transform(df_valid['safe_text'])
X_test = vectorizer.transform(test['safe_text'])

In [23]:
# Define labels
y_train = df_train['label']
y_valid = df_valid['label']

In [24]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

errlgb = []  # Initialize list to store accuracy values
y_pred_totlgb = []  # Initialize list to store total predictions

for fold_, (train_idx, valid_idx) in enumerate(kf.split(X_train)):
    X_train_fold, X_valid_fold = X_train[train_idx], X_train[valid_idx]
    y_train_fold, y_valid_fold = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    # Model
    catboost_classifier = CatBoostClassifier(iterations=8000,
                                            learning_rate=0.055,
                                            depth=8,
                                            eval_metric='Accuracy',  # Changed to accuracy metric
                                            random_seed=42,
                                            bagging_temperature=0.2,
                                            od_type='Iter',
                                            metric_period=50,
                                            od_wait=300)
    
    catboost_classifier.fit(X_train_fold, y_train_fold,
                            use_best_model=True,
                            eval_set=(X_valid_fold, y_valid_fold),  # Provide validation set for early stopping
                            verbose=50)
    
    y_pred_prob = catboost_classifier.predict_proba(X_test)[:, 1]  # Predict class probabilities
    y_pred = np.round(y_pred_prob)  # Convert probabilities to binary predictions
    
    accuracy = accuracy_score(y_valid_fold, catboost_classifier.predict(X_valid_fold))
    print("Accuracy: ", accuracy)
    errlgb.append(accuracy)
    y_pred_totlgb.append(y_pred)

0:	learn: 0.6446320	test: 0.6318750	best: 0.6318750 (0)	total: 1.04s	remaining: 2h 18m 32s
50:	learn: 0.7074543	test: 0.6931250	best: 0.6950000 (49)	total: 40.2s	remaining: 1h 44m 22s
100:	learn: 0.7196437	test: 0.7106250	best: 0.7106250 (94)	total: 1m 17s	remaining: 1h 41m 34s
150:	learn: 0.7321456	test: 0.7100000	best: 0.7112500 (135)	total: 1m 56s	remaining: 1h 40m 40s
200:	learn: 0.7407407	test: 0.7093750	best: 0.7112500 (135)	total: 2m 33s	remaining: 1h 39m 15s
250:	learn: 0.7566807	test: 0.7106250	best: 0.7131250 (230)	total: 3m 11s	remaining: 1h 38m 18s
300:	learn: 0.7632443	test: 0.7106250	best: 0.7131250 (230)	total: 3m 48s	remaining: 1h 37m 23s
350:	learn: 0.7732458	test: 0.7162500	best: 0.7162500 (348)	total: 4m 26s	remaining: 1h 36m 46s
400:	learn: 0.7815284	test: 0.7181250	best: 0.7181250 (365)	total: 5m 3s	remaining: 1h 35m 59s
450:	learn: 0.7866854	test: 0.7162500	best: 0.7181250 (365)	total: 5m 41s	remaining: 1h 35m 22s
500:	learn: 0.7924676	test: 0.7212500	best: 0.7212

0:	learn: 0.6588529	test: 0.6643750	best: 0.6643750 (0)	total: 843ms	remaining: 1h 52m 25s
50:	learn: 0.7038600	test: 0.7187500	best: 0.7212500 (41)	total: 38.7s	remaining: 1h 40m 26s
100:	learn: 0.7157368	test: 0.7200000	best: 0.7231250 (91)	total: 1m 17s	remaining: 1h 40m 49s
150:	learn: 0.7285513	test: 0.7250000	best: 0.7281250 (147)	total: 1m 54s	remaining: 1h 39m 24s
200:	learn: 0.7429286	test: 0.7325000	best: 0.7325000 (200)	total: 2m 31s	remaining: 1h 38m 1s
250:	learn: 0.7557431	test: 0.7337500	best: 0.7350000 (223)	total: 3m 8s	remaining: 1h 36m 58s
300:	learn: 0.7662135	test: 0.7337500	best: 0.7350000 (223)	total: 3m 45s	remaining: 1h 36m 3s
350:	learn: 0.7724644	test: 0.7312500	best: 0.7350000 (223)	total: 4m 22s	remaining: 1h 35m 18s
400:	learn: 0.7823097	test: 0.7325000	best: 0.7350000 (223)	total: 4m 59s	remaining: 1h 34m 33s
450:	learn: 0.7896546	test: 0.7337500	best: 0.7350000 (223)	total: 5m 37s	remaining: 1h 34m 11s
500:	learn: 0.7949680	test: 0.7337500	best: 0.735000

0:	learn: 0.6508830	test: 0.6268750	best: 0.6268750 (0)	total: 780ms	remaining: 1h 44m 2s
50:	learn: 0.7077668	test: 0.6893750	best: 0.6918750 (48)	total: 38.2s	remaining: 1h 39m 10s
100:	learn: 0.7230817	test: 0.7012500	best: 0.7025000 (91)	total: 1m 14s	remaining: 1h 37m 35s
150:	learn: 0.7337084	test: 0.7093750	best: 0.7093750 (150)	total: 1m 51s	remaining: 1h 36m 37s
200:	learn: 0.7441788	test: 0.7131250	best: 0.7156250 (191)	total: 2m 28s	remaining: 1h 36m 7s
250:	learn: 0.7537115	test: 0.7156250	best: 0.7162500 (234)	total: 3m 5s	remaining: 1h 35m 28s
300:	learn: 0.7629317	test: 0.7212500	best: 0.7225000 (286)	total: 3m 42s	remaining: 1h 34m 50s
350:	learn: 0.7730895	test: 0.7262500	best: 0.7262500 (350)	total: 4m 19s	remaining: 1h 34m 7s
400:	learn: 0.7794968	test: 0.7268750	best: 0.7281250 (381)	total: 4m 56s	remaining: 1h 33m 33s
450:	learn: 0.7841850	test: 0.7243750	best: 0.7281250 (381)	total: 5m 33s	remaining: 1h 33m
500:	learn: 0.7935615	test: 0.7268750	best: 0.7281250 (38

0:	learn: 0.6696359	test: 0.6575000	best: 0.6575000 (0)	total: 912ms	remaining: 2h 1m 34s
50:	learn: 0.7027661	test: 0.6862500	best: 0.6868750 (49)	total: 39.8s	remaining: 1h 43m 28s
100:	learn: 0.7204251	test: 0.6918750	best: 0.6937500 (96)	total: 1m 21s	remaining: 1h 45m 35s
150:	learn: 0.7340209	test: 0.6981250	best: 0.7006250 (124)	total: 1m 57s	remaining: 1h 41m 45s
200:	learn: 0.7471480	test: 0.7037500	best: 0.7037500 (186)	total: 2m 31s	remaining: 1h 38m 7s
250:	learn: 0.7583997	test: 0.7050000	best: 0.7075000 (237)	total: 3m 5s	remaining: 1h 35m 39s
300:	learn: 0.7673074	test: 0.7075000	best: 0.7075000 (237)	total: 3m 40s	remaining: 1h 33m 53s
350:	learn: 0.7752774	test: 0.7075000	best: 0.7087500 (304)	total: 4m 14s	remaining: 1h 32m 20s
400:	learn: 0.7805907	test: 0.7062500	best: 0.7087500 (304)	total: 4m 47s	remaining: 1h 30m 54s
450:	learn: 0.7860603	test: 0.7068750	best: 0.7087500 (304)	total: 5m 21s	remaining: 1h 29m 49s
500:	learn: 0.7901235	test: 0.7068750	best: 0.708750

0:	learn: 0.6659375	test: 0.6722952	best: 0.6722952 (0)	total: 905ms	remaining: 2h 36s
50:	learn: 0.7059375	test: 0.6904315	best: 0.6929331 (47)	total: 47.1s	remaining: 2h 2m 27s
100:	learn: 0.7201563	test: 0.6941839	best: 0.6973108 (88)	total: 1m 35s	remaining: 2h 4m 12s
150:	learn: 0.7306250	test: 0.7060663	best: 0.7060663 (142)	total: 2m 19s	remaining: 2h 36s
200:	learn: 0.7414063	test: 0.7079425	best: 0.7079425 (197)	total: 3m 5s	remaining: 1h 59m 40s
250:	learn: 0.7559375	test: 0.7085679	best: 0.7123202 (227)	total: 3m 49s	remaining: 1h 58m 6s
300:	learn: 0.7671875	test: 0.7116948	best: 0.7123202 (227)	total: 4m 32s	remaining: 1h 56m 19s
350:	learn: 0.7756250	test: 0.7110694	best: 0.7123202 (227)	total: 5m 13s	remaining: 1h 53m 43s
400:	learn: 0.7845313	test: 0.7129456	best: 0.7135710 (395)	total: 6m 1s	remaining: 1h 54m 4s
450:	learn: 0.7928125	test: 0.7116948	best: 0.7141964 (420)	total: 6m 48s	remaining: 1h 53m 56s
500:	learn: 0.7979688	test: 0.7116948	best: 0.7141964 (420)	tot

In [25]:
# Perform grid search

# Define evaluation set
eval_set = [(X_valid_fold, y_valid_fold)]

# Define parameter grid for grid search
param_grid = {
    'iterations': [100, 200],
    'learning_rate': [0.01, 0.1],
    'depth': [4, 6],
    'l2_leaf_reg': [1, 3]
}

# Initialize CatBoost classifier
grid_search = GridSearchCV(estimator=catboost_classifier, param_grid=param_grid, cv=3,
                        scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train_fold, y_train_fold, eval_set=eval_set)


Fitting 3 folds for each of 16 candidates, totalling 48 fits


0:	learn: 0.6099391	test: 0.6203877	best: 0.6203877 (0)	total: 85.8ms	remaining: 8.49s
50:	learn: 0.6577590	test: 0.6616635	best: 0.6616635 (49)	total: 3.47s	remaining: 3.33s
99:	learn: 0.6685420	test: 0.6685428	best: 0.6697936 (90)	total: 6.19s	remaining: 0us

bestTest = 0.669793621
bestIteration = 90

Shrink model to first 91 iterations.
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.01; total time=   7.0s


0:	learn: 0.6229201	test: 0.6297686	best: 0.6297686 (0)	total: 60.1ms	remaining: 5.95s
50:	learn: 0.6545582	test: 0.6541588	best: 0.6741714 (2)	total: 2.94s	remaining: 2.83s
99:	learn: 0.6665104	test: 0.6629143	best: 0.6741714 (2)	total: 5.69s	remaining: 0us

bestTest = 0.6741713571
bestIteration = 2

Shrink model to first 3 iterations.
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.01; total time=   6.3s


0:	learn: 0.5615186	test: 0.5590994	best: 0.5590994 (0)	total: 55.8ms	remaining: 5.52s
50:	learn: 0.6667448	test: 0.6535335	best: 0.6535335 (50)	total: 2.89s	remaining: 2.77s
99:	learn: 0.6730724	test: 0.6622889	best: 0.6654159 (72)	total: 5.51s	remaining: 0us

bestTest = 0.6654158849
bestIteration = 72

Shrink model to first 73 iterations.
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.01; total time=   6.2s


0:	learn: 0.6099391	test: 0.6203877	best: 0.6203877 (0)	total: 56.6ms	remaining: 5.6s
50:	learn: 0.7006564	test: 0.6860538	best: 0.6860538 (41)	total: 2.98s	remaining: 2.86s
99:	learn: 0.7208158	test: 0.7054409	best: 0.7060663 (91)	total: 5.75s	remaining: 0us

bestTest = 0.7060662914
bestIteration = 91

Shrink model to first 92 iterations.
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.1; total time=   6.4s


...

0:	learn: 0.6509611	test: 0.6654159	best: 0.6654159 (0)	total: 156ms	remaining: 31.1s
50:	learn: 0.7105016	test: 0.6991870	best: 0.6991870 (50)	total: 8.09s	remaining: 23.6s
100:	learn: 0.7266760	test: 0.7004378	best: 0.7029393 (96)	total: 15.9s	remaining: 15.6s
150:	learn: 0.7531646	test: 0.7054409	best: 0.7060663 (129)	total: 24.8s	remaining: 8.05s
199:	learn: 0.7700422	test: 0.7110694	best: 0.7123202 (190)	total: 34s	remaining: 0us

bestTest = 0.7123202001
bestIteration = 190

Shrink model to first 191 iterations.
[CV] END depth=6, iterations=200, l2_leaf_reg=3, learning_rate=0.1; total time=  34.7s


0:	learn: 0.6348723	test: 0.6404003	best: 0.6404003 (0)	total: 245ms	remaining: 48.7s
50:	learn: 0.7112726	test: 0.6841776	best: 0.6885553 (35)	total: 8.32s	remaining: 24.3s
100:	learn: 0.7309585	test: 0.6929331	best: 0.6948093 (91)	total: 15.4s	remaining: 15.1s
150:	learn: 0.7506445	test: 0.7016886	best: 0.7023139 (143)	total: 22.6s	remaining: 7.33s
199:	learn: 0.7637685	test: 0.7041901	best: 0.7041901 (185)	total: 29.6s	remaining: 0us

bestTest = 0.7041901188
bestIteration = 185

Shrink model to first 186 iterations.
[CV] END depth=6, iterations=200, l2_leaf_reg=3, learning_rate=0.1; total time=  30.5s


0:	learn: 0.6187017	test: 0.6160100	best: 0.6160100 (0)	total: 156ms	remaining: 31s
50:	learn: 0.7136161	test: 0.6904315	best: 0.6916823 (47)	total: 8s	remaining: 23.4s
100:	learn: 0.7354113	test: 0.6960600	best: 0.7010632 (81)	total: 15.6s	remaining: 15.3s
150:	learn: 0.7553316	test: 0.6979362	best: 0.7023139 (121)	total: 23.4s	remaining: 7.6s
199:	learn: 0.7670494	test: 0.7010632	best: 0.7035647 (196)	total: 31.3s	remaining: 0us

bestTest = 0.703564728
bestIteration = 196

Shrink model to first 197 iterations.
[CV] END depth=6, iterations=200, l2_leaf_reg=3, learning_rate=0.1; total time=  32.0s


0:	learn: 0.6543750	test: 0.6604128	best: 0.6604128 (0)	total: 220ms	remaining: 21.8s
50:	learn: 0.7150000	test: 0.7016886	best: 0.7016886 (50)	total: 13.7s	remaining: 13.2s
99:	learn: 0.7356250	test: 0.7079425	best: 0.7085679 (82)	total: 26.9s	remaining: 0us

bestTest = 0.7085678549
bestIteration = 82

Shrink model to first 83 iterations.


GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostClassifier object at 0x000001A251AF6190>,
             param_grid={'depth': [4, 6], 'iterations': [100, 200],
                         'l2_leaf_reg': [1, 3], 'learning_rate': [0.01, 0.1]},
             scoring='neg_mean_squared_error', verbose=2)

In [26]:
#  Print best parameters and corresponding RMSE for each fold
print(f"Fold {fold_ + 1} Best Parameters:", grid_search.best_params_)
print(f"Fold {fold_ + 1} Best RMSE:", np.sqrt(-grid_search.best_score_))


Fold 5 Best Parameters: {'depth': 6, 'iterations': 100, 'l2_leaf_reg': 1, 'learning_rate': 0.1}
Fold 5 Best RMSE: 0.6843128346233013


In [27]:
# Use best model for predictions
best_model = grid_search.best_estimator_
valid_preds_catboost = best_model.predict(X_valid_fold)

In [28]:
# Calculate RMSE on the validation set for each fold
rmse_catboost = np.sqrt(mean_squared_error(y_valid_fold, valid_preds_catboost))
print(f"Fold {fold_ + 1} Validation RMSE (CatBoost):", rmse_catboost)


NameError: name 'mean_squared_error' is not defined

In [ ]:
# Make predictions on the test set for each fold
test_preds_catboost = best_model.predict(X_test)

In [ ]:
# Make a submission dataframe for CatBoost for each fold (optional)
sub_catboost = pd.DataFrame({
        'tweet_id': test['tweet_id'],
        'label': test_preds_catboost.reshape(-1)
    })
sub_catboost.to_csv(f'catboost_submission_gridsearch_fold_{fold_ + 1}.csv', index=False)
sub_catboost.head()
